In [ ]:
import sys
sys.float_info
#Should reveal that floats have 8 bytes

In [ ]:
MD = "metaData"
Indep = "trainIndependents"
Dep = "trainDependent"
fileNames = [MD, Indep, Dep]

In [ ]:
import math ##For basic mathematical operations
import matplotlib.pyplot as plt
import dask.dataframe as dd  
from IPython import display ## Plot setup for Ipython
from matplotlib import cm ##  Colormap reference
from matplotlib import gridspec ##plot setups
from matplotlib import pyplot as plt ##plot setups
import numpy as np 
import pandas as pd
from sklearn.linear_model import LinearRegression
from tqdm import tqdm
import collections
import warnings
# warnings.filterwarnings("ignore") - may want to ignore, if know what you are doing, but can be helpful
# warnings.filterwarnings(action = "once") - may want to show only once
# from Queue import priority_queue

In [ ]:
#takes about 1:30 for me
frames = []

for index, fileName in tqdm(enumerate(fileNames)):
    frames.append(dd.read_csv("./TestData/" + fileName + ".csv", sep=",", sample = 2000000))

In [ ]:
#about 8 seconds
for i in tqdm(range(len(frames))):
    frames[i] = frames[i].fillna(frames[i].mean())

In [ ]:
#15 seco
frames[-1].compute()